In [1]:
import pandas as pd
import re
import numpy as np
from datetime import datetime

res_df = pd.read_csv('Restaurant_Data\Restaurant_Tripadvisor_Data_SEA.csv')

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_2452\1682315425.py:6: DtypeWarning: Columns (7,11,20) have mixed types. Specify dtype option on import or set low_memory=False.
  res_df = pd.read_csv('Restaurant_Data\Restaurant_Tripadvisor_Data_SEA.csv')


In [2]:
res_df.duplicated().sum()

0

In [3]:
res_df.drop_duplicates()

,Restaurant ID,Country,Tourist Attraction,Restaurant Name,Address,Restaurant Website,Hotline,Ranking,Price Type,Opening and Closing hours,...,Poor Reviews,Terrible Reviews,Description,Cuisines,Meal,Special Diets,Features,Nearby Hotels,Nearby Restaurants,Nearby Attractions
0,RS000001,Vietnam,Ha Noi,Co Vietnamese Restaurant & Vegan,"21 Hàng Cân, Hoan Kiem, Hanoi 10000 Vietnam",https://covietnameserestaurant.com/,+84 94 633 55 88,#91 of 136 Restaurants in Hoan Kiem,$,Sun:10:00 AM - 11:00 PM;Mon:10:00 AM - 11:00 P...,...,1.0,3.0,"A distinctive, well-preserved and comfortable ...","Asian, Deli, Vietnamese, Healthy, Indigenous","Lunch, Dinner, Brunch, Drinks","Vegetarian friendly, Vegan options",Accepts Credit Cards|American Express|Digital ...,"La Siesta Premium Hang Be Hotel(5.0/3,313–0.3 ...","Met Vietnamese Restaurant & Vegan 3(5.0/1,608–...",D&c Art Gallery(None/None–0.01 miles away)|Tig...
1,RS000002,Vietnam,Ha Noi,Vie L'essence De Cuisine,"99B Hang Gai St, Hoan Kiem, Hanoi 11015 Vietnam",https://vielessencedecuisine.com/contact-us/,+84 98 825 62 26,#33 of 136 Restaurants in Hoan Kiem,$$ - $$$,Sun:10:30 AM - 10:00 PM;Mon:10:30 AM - 10:00 P...,...,1.0,1.0,Vie L'essence De Cuisine là nhà hàng đầu tư ch...,"Asian, Vietnamese, Healthy, Wine Bar, Indigenous","Lunch, Dinner, Brunch, Drinks","Vegan options, Gluten free options",Accepts Credit Cards|American Express|Buffet|D...,"The Oriental Jade Hotel(5.0/4,693–0.14 miles a...",Mẹt Vietnamese Restaurant & Vegetarian Food 5(...,Hanoi Old Quarter Vietnam(4.0/606–0 miles away...
2,RS000003,Vietnam,Ha Noi,Aira Sky Bar & Restaurant,38A Tran Phu St AIRA Boutique Hanoi Hotel & Sp...,https://essencedining.com/,+84 24 3935 2485,#4 of 33 Restaurants in Ba Dinh,$$ - $$$,Sun:4:30 PM - 11:00 PM;Mon:4:30 PM - 11:00 PM;...,...,1.0,0.0,Whether you’re in the mood for a happy hour co...,"Bar, International, Dining bars","Breakfast, Lunch, Dinner, Late Night, Drinks",NaN,Accepts Credit Cards|Free Wifi|Full Bar|Outdoo...,"Meritel Hanoi(5.0/1,777–0.25 miles away)|La Si...",Essence Restaurant Original (since 2011)(5.0/6...,Rabbit Habit(5.0/1–0.03 miles away)|B9 Studio(...
3,RS000004,Vietnam,Ha Noi,Cau Go Vietnamese Cuisine Restaurant,"No 9 Dinh Tien Hoang Street, Hoan Kiem, Hanoi ...",http://www.caugorestaurant.com/,+84 24 3926 0808,#86 of 136 Restaurants in Hoan Kiem,$$ - $$$,Sun:10:00 AM - 10:00 PM;Mon:10:00 AM - 10:00 P...,...,80.0,56.0,"Nestled in the heart of Hanoi, Cau Go Vietname...","Asian, Vietnamese, Healthy","Lunch, Dinner, Brunch, Late Night","Vegetarian friendly, Vegan options, Gluten fre...",Accepts Credit Cards|Free Wifi|Full Bar|Highch...,"The Oriental Jade Hotel(5.0/4,693–0.21 miles a...",The Majesty Sky Bar(5.0/507–0.09 miles away)|T...,"Thang Long Water Puppet(4.0/12,400–0.08 miles ..."
4,RS000005,Vietnam,Ha Noi,Quan Que Restaurant,"80 P. Mã Mây, Hoan Kiem, Hanoi 100000 Vietnam",http://quanquerestaurant.vn,+84 91 640 08 58,#38 of 136 Restaurants in Hoan Kiem,$,Sun:10:00 AM - 11:00 PM;Mon:10:00 AM - 11:00 P...,...,15.0,10.0,"Located at the 80 Mã Mây St, one of the most o...","Cafe, Asian, Vietnamese, Healthy","Lunch, Dinner, Brunch, Late Night, Drinks","Vegetarian friendly, Vegan options, Gluten fre...",Accepts Credit Cards|Free off-street parking|F...,"La Siesta Premium Hang Be Hotel(5.0/3,313–0.11...","Era Restaurant(5.0/2,877–0.06 miles away)|Diam...",Ancient House(4.2/516–0.01 miles away)|Fe!n Cl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131241,RS131242,Singapore,Singapore,Jiang Cantonese Tapas Restaurant,NaN,http://www.jiangcantonesetapas.com,+65 6518 4728,NaN,NaN,Sun:nan;Mon:nan;Tue:nan;Wed:nan;Thu:nan;Fri:na...,...,NaN,NaN,Chinese,Chinese,NaN,NaN,NaN,NaN,NaN,NaN
131242,RS131243,Singapore,Singapore,Uozen Singapore,NaN,https://www.illius.sg,+65 6636 0935,NaN,NaN,Sun:nan;Mon:nan;Tue:nan;Wed:nan;Thu:nan;Fri:na...,...,NaN,NaN,Japanese,Japanese,NaN,NaN,NaN,NaN,NaN,NaN
131243,RS131244,Singapore,Singapore,Eddy Western Delight,NaN,NaN,NaN,Na

In [4]:
def missingCounter(df):
    missing_data = df.isna().sum()
    missing_data = missing_data[missing_data > 0].sort_values(ascending = False )
    print(missing_data )
    return missing_data.index
missingCounter(res_df)

Special Diets         112542
Features               77939
Price Type             67838
Meal                   52661
Restaurant Website     49066
Average Reviews        30838
Rating                 30838
Reviewers              30838
Excellent Reviews      30838
Good Reviews           30838
Poor Reviews           30838
Terrible Reviews       30838
Ranking                30406
Hotline                21409
Cuisines               20736
Description            11102
Nearby Attractions      9574
Nearby Restaurants      9394
Nearby Hotels           7827
Address                 7187
Restaurant Name            7
dtype: int64


Index(['Special Diets', 'Features', 'Price Type', 'Meal', 'Restaurant Website',
       'Average Reviews', 'Rating', 'Reviewers', 'Excellent Reviews',
       'Good Reviews', 'Poor Reviews', 'Terrible Reviews', 'Ranking',
       'Hotline', 'Cuisines', 'Description', 'Nearby Attractions',
       'Nearby Restaurants', 'Nearby Hotels', 'Address', 'Restaurant Name'],
      dtype='object')

In [5]:
res_df.drop(columns=['Nearby Attractions', 'Nearby Restaurants', 'Nearby Hotels'], inplace=True)

In [6]:
import unicodedata
# Xóa các dòng có giá trị NaN trong cột 'Name'
res_df = res_df.dropna(subset=['Restaurant Name'])

# Hàm chuyển chuỗi có dấu sang không dấu
def remove_diacritics(text):
    if pd.isna(text): return ''
    text = unicodedata.normalize('NFD', str(text))
    text = ''.join(c for c in text if unicodedata.category(c) != 'Mn')
    text = text.replace('Đ', 'D').replace('đ', 'd')  # Xử lý đặc biệt
    return text


# Làm sạch cột 'Name'
def clean_name(text):
    if pd.isna(text): return ''
    text = remove_diacritics(text)
    text = text.replace('Unclaimed', '').strip()
    return text

# Làm sạch địa chỉ: bỏ dấu + bỏ mã số đứng trước tên quốc gia
def clean_address(text):
    if pd.isna(text): return ''
    text = remove_diacritics(text)

    # Bước 1: Xóa mã vùng kiểu "20000", "310000", v.v. trước tên quốc gia
    pattern = r'\s\d{4,6}(?=\s*(Vietnam|Thailand|Philippines|Malaysia|Singapore|Indonesia|Cambodia|Laos|Myanmar|Brunei|Timor))'
    text = re.sub(pattern, '', text, flags=re.IGNORECASE)

    # Bước 2: Thêm dấu "," nếu không có dấu phẩy ngay trước tên quốc gia
    text = re.sub(r'(?<![,])\s+(?=(Vietnam|Thailand|Philippines|Malaysia|Singapore|Indonesia|Cambodia|Laos|Myanmar|Brunei|Timor)$)', r', ', text.strip(), flags=re.IGNORECASE)

    return text.strip()


res_df['Restaurant Name'] = res_df['Restaurant Name'].apply(clean_name)
res_df['Address'] = res_df['Address'].apply(clean_address)


In [7]:
res_df

,Restaurant ID,Country,Tourist Attraction,Restaurant Name,Address,Restaurant Website,Hotline,Ranking,Price Type,Opening and Closing hours,...,Excellent Reviews,Good Reviews,Average Reviews,Poor Reviews,Terrible Reviews,Description,Cuisines,Meal,Special Diets,Features
0,RS000001,Vietnam,Ha Noi,Co Vietnamese Restaurant & Vegan,"21 Hang Can, Hoan Kiem, Hanoi, Vietnam",https://covietnameserestaurant.com/,+84 94 633 55 88,#91 of 136 Restaurants in Hoan Kiem,$,Sun:10:00 AM - 11:00 PM;Mon:10:00 AM - 11:00 P...,...,448.0,13.0,5.0,1.0,3.0,"A distinctive, well-preserved and comfortable ...","Asian, Deli, Vietnamese, Healthy, Indigenous","Lunch, Dinner, Brunch, Drinks","Vegetarian friendly, Vegan options",Accepts Credit Cards|American Express|Digital ...
1,RS000002,Vietnam,Ha Noi,Vie L'essence De Cuisine,"99B Hang Gai St, Hoan Kiem, Hanoi, Vietnam",https://vielessencedecuisine.com/contact-us/,+84 98 825 62 26,#33 of 136 Restaurants in Hoan Kiem,$$ - $$$,Sun:10:30 AM - 10:00 PM;Mon:10:30 AM - 10:00 P...,...,1090.0,7.0,5.0,1.0,1.0,Vie L'essence De Cuisine là nhà hàng đầu tư ch...,"Asian, Vietnamese, Healthy, Wine Bar, Indigenous","Lunch, Dinner, Brunch, Drinks","Vegan options, Gluten free options",Accepts Credit Cards|American Express|Buffet|D...
2,RS000003,Vietnam,Ha Noi,Aira Sky Bar & Restaurant,38A Tran Phu St AIRA Boutique Hanoi Hotel & Sp...,https://essencedining.com/,+84 24 3935 2485,#4 of 33 Restaurants in Ba Dinh,$$ - $$$,Sun:4:30 PM - 11:00 PM;Mon:4:30 PM - 11:00 PM;...,...,698.0,12.0,0.0,1.0,0.0,Whether you’re in the mood for a happy hour co...,"Bar, International, Dining bars","Breakfast, Lunch, Dinner, Late Night, Drinks",NaN,Accepts Credit Cards|Free Wifi|Full Bar|Outdoo...
3,RS000004,Vietnam,Ha Noi,Cau Go Vietnamese Cuisine Restaurant,"No 9 Dinh Tien Hoang Street, Hoan Kiem, Hanoi,...",http://www.caugorestaurant.com/,+84 24 3926 0808,#86 of 136 Restaurants in Hoan Kiem,$$ - $$$,Sun:10:00 AM - 10:00 PM;Mon:10:00 AM - 10:00 P...,...,1808.0,584.0,177.0,80.0,56.0,"Nestled in the heart of Hanoi, Cau Go Vietname...","Asian, Vietnamese, Healthy","Lunch, Dinner, Brunch, Late Night","Vegetarian friendly, Vegan options, Gluten fre...",Accepts Credit Cards|Free Wifi|Full Bar|Highch...
4,RS000005,Vietnam,Ha Noi,Quan Que Restaurant,"80 P. Ma May, Hoan Kiem, Hanoi, Vietnam",http://quanquerestaurant.vn,+84 91 640 08 58,#38 of 136 Restaurants in Hoan Kiem,$,Sun:10:00 AM - 11:00 PM;Mon:10:00 AM - 11:00 P...,...,2256.0,136.0,14.0,15.0,10.0,"Located at the 80 Mã Mây St, one of the most o...","Cafe, Asian, Vietnamese, Healthy","Lunch, Dinner, Brunch, Late Night, Drinks","Vegetarian friendly, Vegan options, Gluten fre...",Accepts Credit Cards|Free off-street parking|F...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131241,RS131242,Singapore,Singapore,Jiang Cantonese Tapas Restaurant,,http://www.jiangcantonesetapas.com,+65 6518 4728,NaN,NaN,Sun:nan;Mon:nan;Tue:nan;Wed:nan;Thu:nan;Fri:na...,...,NaN,NaN,NaN,NaN,NaN,Chinese,Chinese,NaN,NaN,NaN
131242,RS131243,Singapore,Singapore,Uozen Singapore,,https://www.illius.sg,+65 6636 0935,NaN,NaN,Sun:nan;Mon:nan;Tue:nan;Wed:nan;Thu:nan;Fri:na...,...,NaN,NaN,NaN,NaN,NaN,Japanese,Japanese,NaN,NaN,NaN
131243,RS131244,Singapore,Singapore,Eddy Western Delight,,NaN,NaN,NaN,NaN,Sun:nan;Mon:nan;Tue:nan;Wed:nan;Thu:nan;Fri:na...,...,NaN,NaN,NaN,NaN,NaN,American,American,NaN,NaN,NaN
131244,RS131245,Singapore,Singapore,Hongdae Korean Bbq,"118 Neil Road, Singapore, Singapore",NaN,6582990844,NaN,NaN,Sun:11:30 AM - 11:30 PM;Mon:11:30 AM - 11:30 P...,...,NaN,NaN,NaN,NaN,NaN,"Lunch, Dinner","Barbecue, Asian, Korean, Grill","Lunch, Dinner",NaN,NaN


In [8]:
def parse_ranking(ranking_text):
    try:
        match = re.match(r"#(\d+)\s+of\s+(\d+)\s+Restaurants\s+in\s+(.+)", str(ranking_text))
        if match:
            position = int(match.group(1))
            total = int(match.group(2))
            area = match.group(3).strip()
            return pd.Series([position, total, area])
    except:
        pass
    # Nếu không parse được, gán giá trị mặc định
    return pd.Series([-1, -1, "Unknown"])

# Áp dụng tách
res_df[['Ranking_Position', 'Ranking_Total', 'Ranking_Area']] = res_df['Ranking'].apply(parse_ranking)

In [9]:
res_df.drop(columns=['Ranking'], inplace=True)

In [10]:
res_df['have_website'] = res_df['Restaurant Website'].notnull()
res_df['have_hotline'] = res_df['Hotline'].notnull()
res_df['have_description'] = res_df['Description'].notnull()


In [11]:
res_df = res_df.drop(columns=['Restaurant Website', "Hotline", 'Description'])

In [12]:
def normalize_price(price_type):
    # Nếu là NaN (float), trả về luôn "No price category"
    if not isinstance(price_type, str):
        return "No price category"
    price_type = price_type.strip()
    if price_type == "$":
        return "Cheap Eats"
    elif price_type == "$$ - $$$":
        return "Mid Range"
    elif price_type == "$$$$":
        return "Fine Dining"
    else:
        return "No price category"

res_df['Price_Category'] = res_df['Price Type'].apply(normalize_price)


In [13]:
res_df = res_df.drop(columns=['Price Type'])

In [14]:

# Các cột review cần fill
review_cols = [
    'Reviewers',
    'Excellent Reviews',
    'Good Reviews',
    'Average Reviews',
    'Poor Reviews',
    'Terrible Reviews'
]

# Fill NaN hoặc giá trị thiếu thành 0 trên cột gốc
for col in review_cols:
    res_df[col] = res_df[col].fillna(0)


# (Sau đó bạn tiếp tục dùng các bước làm sạch, chuẩn hóa, tính toán tỷ lệ như các script mình đã gửi trước đó.)

# Ví dụ: tiếp tục chuyển sang số và làm sạch nếu cần
def to_numeric(val):
    try:
        return float(str(val).replace(',', '').strip())
    except:
        return 0

for col in review_cols:
    res_df[col] = res_df[col].apply(to_numeric)

# Fill missing rating = -1
res_df['Rating'] = res_df['Rating'].fillna(-1)



In [15]:
# 1. Làm sạch các cột review gốc (không tạo cột mới!)
def clean_and_fill_value(val):
    if pd.isna(val):
        return 0
    try:
        return float(str(val).replace(',', '').strip())
    except:
        return 0

rating_columns = [
    'Reviewers',
    'Excellent Reviews',
    'Good Reviews',
    'Average Reviews',
    'Poor Reviews',
    'Terrible Reviews'
]
for col in rating_columns:
    res_df[col] = res_df[col].apply(clean_and_fill_value)

# 2. Tính phần trăm từng loại review
def safe_divide(a, b):
    if b == 0:
        return 0
    return a / b

res_df['Pct_Excellent'] = res_df.apply(lambda row: safe_divide(row['Excellent Reviews'], row['Reviewers']), axis=1)
res_df['Pct_Good'] = res_df.apply(lambda row: safe_divide(row['Good Reviews'], row['Reviewers']), axis=1)
res_df['Pct_Average'] = res_df.apply(lambda row: safe_divide(row['Average Reviews'], row['Reviewers']), axis=1)
res_df['Pct_Poor'] = res_df.apply(lambda row: safe_divide(row['Poor Reviews'], row['Reviewers']), axis=1)
res_df['Pct_Terrible'] = res_df.apply(lambda row: safe_divide(row['Terrible Reviews'], row['Reviewers']), axis=1)

def safe_score(row):
    if row['Reviewers'] == 0:
        return 0
    return (
        5 * row['Excellent Reviews'] +
        4 * row['Good Reviews'] +
        3 * row['Average Reviews'] +
        2 * row['Poor Reviews'] +
        1 * row['Terrible Reviews']
    ) / row['Reviewers']

res_df['Review_Score'] = res_df.apply(safe_score, axis=1)

In [16]:
list_cols = ['Special Diets', 'Features', 'Meal', 'Cuisines']

def safe_split(val):
    # Nếu bị thiếu dữ liệu => trả về list rỗng
    if pd.isna(val) or str(val).strip() == '':
        return []
    # Split bằng dấu phẩy, strip từng phần tử
    return [x.strip().lower() for x in str(val).split(',') if x.strip()]

for col in list_cols:
    # Tạo cột mới chứa list đã chuẩn hóa
    res_df[f'{col}_list'] = res_df[col].apply(safe_split)

In [17]:
res_df = res_df.drop(columns=['Special Diets', 'Features', 'Meal', 'Cuisines'])

In [18]:
res_df.to_csv('Restaurant_Data\Restaurant_Tripadvisor_Data_SEA_cleaned.csv', index=False, encoding='utf-8', quoting=1)


In [19]:
import ast

# 1. Đọc dữ liệu đã clean
df = pd.read_csv("Restaurant_Data\\Restaurant_Tripadvisor_Data_SEA_cleaned.csv")

# 2. Chuẩn hóa & explode trường features_list
# Nếu trường là chuỗi kiểu "['a', 'b']" thì dùng ast.literal_eval để convert về list
def parse_list(val):
    if pd.isna(val) or val in ['[]', '', 'nan']:
        return []
    try:
        return ast.literal_eval(val)
    except:
        return []
    
df['features_parsed'] = df['Features_list'].apply(parse_list)

# 3. Lấy unique features cho bảng dim_feature
features_all = set()
for feats in df['features_parsed']:
    for f in str(feats).replace('|', ',').replace('[','').replace(']','').replace("'",'').split(','):
        f = f.strip().lower()
        if f and f != 'nan':
            features_all.add(f)
            
features_list = sorted(list(features_all))
dim_feature = pd.DataFrame({'feature_name': features_list})
dim_feature['feature_sk'] = dim_feature.index + 1  # surrogate key bắt đầu từ 1

# 4. Tạo bảng bridge_restaurant_feature
bridge_rows = []
for idx, row in df.iterrows():
    # Tạo surrogate key nhà hàng (bạn sẽ join lại sang dim_restaurant, ở đây dùng chỉ số dòng +1 tạm)
    restaurant_sk = idx + 1
    # Nếu bạn đã mapping restaurant_sk rồi thì dùng key thực tế
    for f in row['features_parsed']:
        # Có thể là chuỗi 'wifi|takeout|seating'
        for feat in str(f).split('|'):
            feat = feat.strip().lower()
            if feat and feat != 'nan':
                feature_sk = dim_feature.loc[dim_feature['feature_name'] == feat, 'feature_sk'].values[0]
                bridge_rows.append({'restaurant_sk': restaurant_sk, 'feature_sk': feature_sk})

bridge_restaurant_feature = pd.DataFrame(bridge_rows)

# 5. Xuất ra file để import vào Oracle
dim_feature[['feature_sk', 'feature_name']].to_csv('Restaurant_Data\\dim_feature.csv', index=False)
bridge_restaurant_feature.to_csv('Restaurant_Data\\bridge_restaurant_feature.csv', index=False)



In [20]:
import pandas as pd
import ast

df = pd.read_csv("Restaurant_Data\\Restaurant_Tripadvisor_Data_SEA_cleaned.csv")

def parse_list(val):
    if pd.isna(val) or val in ['[]', '', 'nan']:
        return []
    try:
        return ast.literal_eval(val)
    except:
        return []

# ------ 1. Cuisines ---------
df['cuisines_parsed'] = df['Cuisines_list'].apply(parse_list)
cuisines_all = set()
for cuis in df['cuisines_parsed']:
    for c in cuis:
        c = str(c).strip().lower()
        if c and c != 'nan':
            cuisines_all.add(c)
dim_cuisine = pd.DataFrame({'cuisine_name': sorted(list(cuisines_all))})
dim_cuisine['cuisine_sk'] = dim_cuisine.index + 1

bridge_rows = []
for idx, row in df.iterrows():
    restaurant_sk = idx + 1
    for c in row['cuisines_parsed']:
        c = str(c).strip().lower()
        if c and c != 'nan':
            cuisine_sk = dim_cuisine.loc[dim_cuisine['cuisine_name'] == c, 'cuisine_sk'].values[0]
            bridge_rows.append({'restaurant_sk': restaurant_sk, 'cuisine_sk': cuisine_sk})
bridge_restaurant_cuisine = pd.DataFrame(bridge_rows)

dim_cuisine[['cuisine_sk', 'cuisine_name']].to_csv('Restaurant_Data\\dim_cuisine.csv', index=False)
bridge_restaurant_cuisine.to_csv('Restaurant_Data\\bridge_restaurant_cuisine.csv', index=False)

# ------ 2. Meal ---------
df['meal_parsed'] = df['Meal_list'].apply(parse_list)
meals_all = set()
for meal in df['meal_parsed']:
    for m in meal:
        m = str(m).strip().lower()
        if m and m != 'nan':
            meals_all.add(m)
dim_meal = pd.DataFrame({'meal_name': sorted(list(meals_all))})
dim_meal['meal_sk'] = dim_meal.index + 1

bridge_rows = []
for idx, row in df.iterrows():
    restaurant_sk = idx + 1
    for m in row['meal_parsed']:
        m = str(m).strip().lower()
        if m and m != 'nan':
            meal_sk = dim_meal.loc[dim_meal['meal_name'] == m, 'meal_sk'].values[0]
            bridge_rows.append({'restaurant_sk': restaurant_sk, 'meal_sk': meal_sk})
bridge_restaurant_meal = pd.DataFrame(bridge_rows)

dim_meal[['meal_sk', 'meal_name']].to_csv('Restaurant_Data\\dim_meal.csv', index=False)
bridge_restaurant_meal.to_csv('Restaurant_Data\\bridge_restaurant_meal.csv', index=False)

# ------ 3. Special Diets ---------
df['diet_parsed'] = df['Special Diets_list'].apply(parse_list)
diets_all = set()
for diet in df['diet_parsed']:
    for d in diet:
        d = str(d).strip().lower()
        if d and d != 'nan':
            diets_all.add(d)
dim_diet = pd.DataFrame({'diet_name': sorted(list(diets_all))})
dim_diet['diet_sk'] = dim_diet.index + 1

bridge_rows = []
for idx, row in df.iterrows():
    restaurant_sk = idx + 1
    for d in row['diet_parsed']:
        d = str(d).strip().lower()
        if d and d != 'nan':
            diet_sk = dim_diet.loc[dim_diet['diet_name'] == d, 'diet_sk'].values[0]
            bridge_rows.append({'restaurant_sk': restaurant_sk, 'diet_sk': diet_sk})
bridge_restaurant_diet = pd.DataFrame(bridge_rows)

dim_diet[['diet_sk', 'diet_name']].to_csv('Restaurant_Data\\dim_diet.csv', index=False)
bridge_restaurant_diet.to_csv('Restaurant_Data\\bridge_restaurant_diet.csv', index=False)

